In [ ]:
%cd ~/REVIVAL2
%load_ext autoreload
%autoreload 2
%load_ext blackcellmagic

In [2]:
import json
import os

def load_json(file_path):
    """Load JSON content from a file."""
    with open(file_path, "r") as f:
        return json.load(f)

def compare_fields(json1, json2, fields_to_compare):
    """Compare specific fields between two JSON files."""
    differences = {}
    for field in fields_to_compare:
        value1 = json1.get(field, None)
        value2 = json2.get(field, None)

        if value1 != value2:
            differences[field] = {"file1": value1, "file2": value2}

    return differences

# Directory containing JSON files
# json_dir = "/path/to/json/files"
fields_to_compare = ["modifications", "unpairedMsa", "pairedMsa", "templates"]  # Add fields to compare


# Paths to the output JSON files
json_file1 = "/disk2/fli/REVIVAL2/zs/af3_joint/struct/PfTrpB-5iodo/i165di183h/i165di183h_data.json"
# json_file2 = "/disk2/fli/REVIVAL2/zs/af3_joint/struct/PfTrpB-5iodo/y301l/y301l_data.json"
json_file2 = "/disk2/fli/REVIVAL2/zs/af3_joint/struct/PfTrpB-5iodo/i165gi183vy301e/i165gi183vy301e_data.json"

# Get all JSON files in the directory
json_files = [json_file1, json_file2]

# Pairwise comparison of all JSON files
for i in range(len(json_files)):
    for j in range(i + 1, len(json_files)):
        json_file1 = json_file1
        json_file2 = json_file2

        # Load JSON data
        json_data1 = load_json(json_file1)
        json_data2 = load_json(json_file2)

        # Compare fields
        differences = compare_fields(json_data1, json_data2, fields_to_compare)

        # Print results
        if differences:
            print(f"Comparing {json_files[i]} and {json_files[j]}:")
            for field, diff in differences.items():
                print(f"  Field: {field}")
                print(f"    {json_files[i]}: {diff['file1']}")
                print(f"    {json_files[j]}: {diff['file2']}")
        else:
            print(f"{json_files[i]} and {json_files[j]} have the same fields for {fields_to_compare}.")


/disk2/fli/REVIVAL2/zs/af3_joint/struct/PfTrpB-5iodo/i165di183h/i165di183h_data.json and /disk2/fli/REVIVAL2/zs/af3_joint/struct/PfTrpB-5iodo/i165gi183vy301e/i165gi183vy301e_data.json have the same fields for ['modifications', 'unpairedMsa', 'pairedMsa', 'templates'].
